<a href="https://colab.research.google.com/github/ivan-mihailov/LS-Unit-2-Sprint-2-Assignments/blob/main/Ivan_Mihailov_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [5]:
# Split train into train & val
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [6]:
def wrangle(df):
    """Wrangle train, validate, and test sets in the same way"""
    
    # Prevent SettingWithCopyWarning
    df = df.copy()
    
    # Set 'id' as the index for the dataframe
    df = df.set_index('id')

    # Replace near-zero latitude values with zero,
    df['latitude'] = df['latitude'].replace(-2e-08, 0)
    
    # Replace zero values with nulls to prepare for imputation
    # Create a "missing indicator" column, to account for possibility that 
    # missing values may be predictive
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    for col in cols_with_zeros:
        df[col] = df[col].replace(0, np.nan)
        df[col+'_MISSING'] = df[col].isnull()
            
    # Drop duplicate columns
    duplicates = ['quantity_group', 'payment_type']
    df = df.drop(columns=duplicates)
    
    # Drop recorded_by (never varies)
    unusable_variance = ['recorded_by']
    df = df.drop(columns=unusable_variance)
    
    # Convert date_recorded to datetime
    df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    df['year_recorded'] = df['date_recorded'].dt.year
    df['month_recorded'] = df['date_recorded'].dt.month
    df['day_recorded'] = df['date_recorded'].dt.day
    df = df.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    df['years'] = df['year_recorded'] - df['construction_year']
    df['years_MISSING'] = df['years'].isnull()
    
    # return the wrangled dataframe
    return df

In [7]:
# Use wrangle function to clean training, validation, and testing sets the same way
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

In [14]:
# Set up 'status_group' as target
target = 'status_group'

# Set up features as all column except for the target
features = train.drop(columns = [target]).columns.tolist()

In [16]:
# Set up training/validation/testing sets into X feature matrix and y target vector
X_train =  train[features]
y_train = train[target]

X_val =  val[features]
y_val = val[target]

X_test =  test[features]

In [15]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from scipy.stats import randint, uniform

In [19]:
# Set up pipeline with 2 transformers and 1 estimator:
# (1) OrdinalEncoder for all categorical columns, (2) SimpleImputer, and
# (3) Perform RandomForest Classifier on the dataset

rf_model = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)

In [21]:
# Set up parameters distributions as follows: 
# (1) SimpleImputer strategy (mean or median);
# (2) RandomForestClassifier for number of trees, criterion, maximum tree depth, 
# maximum number of features, whether bootstrapping is used & the maximum number
# of samples used when bootstrapping

param_distributions = {
    'simpleimputer__strategy' : ['mean', 'median'],
    'randomforestclassifier__n_estimators' : randint(75, 300),
    'randomforestclassifier__criterion' : ['gini', 'entropy'],
    'randomforestclassifier__max_depth' : [5, 10, 15, 20, 25],
    'randomforestclassifier__max_features' : uniform(0, 1),
    'randomforestclassifier__bootstrap' : ['True', 'False'],
    'randomforestclassifier__max_samples' : uniform(0, 1)
}

# Set up a Randomized Search

search = RandomizedSearchCV(
    rf_model,
    param_distributions = param_distributions,
    n_iter = 7,
    scoring = 'accuracy',
    verbose = 10,
    cv = 3,
    n_jobs = -1,
    refit = True,
    return_train_score = True
)

search.fit(X_train, y_train)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  4.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  4.4min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [23]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation Accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__bootstrap': 'True', 'randomforestclassifier__criterion': 'entropy', 'randomforestclassifier__max_depth': 25, 'randomforestclassifier__max_features': 0.097025749495811, 'randomforestclassifier__max_samples': 0.1743765308324955, 'randomforestclassifier__n_estimators': 195, 'simpleimputer__strategy': 'median'}
Cross-validation Accuracy 0.7888047138047138


In [24]:
val_pipeline = search.best_estimator_ # Set up pipeline for validation set using the Randomized Search optimized hyperparameters

In [25]:
from sklearn.metrics import accuracy_score

y_pred_val = val_pipeline.predict(X_val) # Predict target vector for the validation set using the Randomized Search optimized hyperparameters

val_acc = accuracy_score(y_val, y_pred_val) # Calculate the validation set accuracy score using the Randomized Search optimized hyperparameters

print(f'Accuracy Score for Validation Set is: {val_acc}')

Accuracy Score for Validation Set is: 0.8017676767676768


In [26]:
test_pipeline = search.best_estimator_ # Set up pipeline for test set using the Randomized Search optimized hyperparameters


In [27]:
y_pred_test = test_pipeline.predict(X_test) # Predict target vector for the test set using the Randomized Search optimized hyperparameters

submission = pd.DataFrame(data = y_pred_test, index = X_test.index, columns = ['status_group']) # Assign the predicted values to a dataframe
submission.head()

,status_group
id,
50785,non functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [28]:
# Create Kaggle Submission file
submission.to_csv('Ivan_Mihailov_Module3_Kaggle_Submission')

# IIM Stretch Goals

In [49]:
# Set up parameters distributions fro Grid Search as follows: 
# (1) RandomForestClassifier for maximum number of features

param_distributions = {
    'randomforestclassifier__max_features' : np.arange(1, len(X_train.columns)+1)    
 }

# Set up a GridSearch

search_grid = GridSearchCV(
    rf_model,
    param_grid = param_distributions,
    scoring = 'accuracy',
    verbose = 10,
    cv = 3,
    n_jobs = -1,
    refit = True,
    return_train_score = True
)

search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 45 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed: 15.2min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 20.2min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 26.3min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 33.7min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 135 out of 135 | elapsed: 49

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=None,
                                                       drop_invariant=False,
                                                       handle_missing='value',
                                                       handle_unknown='value',
                                                       mapping=None,
                                                       return_df=True,
                                                       verbose=0)),
                                       ('simpleimputer',
                                        SimpleImputer(add_indicator=False,
                                                      copy=True,
                                                      fill_value=None,
                                                      missing_values=nan,
          

In [50]:
print('Best GridSearch hyperparameters', search_grid.best_params_)
print('Cross-validation GridSearch Accuracy', search_grid.best_score_)

Best GridSearch hyperparameters {'randomforestclassifier__max_features': 9}
Cross-validation GridSearch Accuracy 0.8010101010101011


In [51]:
val_grid_pipeline = search_grid.best_estimator_ # Set up pipeline for validation set for GridSearch optimized hyperparameters

In [52]:
y_pred_val_grid = val_grid_pipeline.predict(X_val) # Predict target vector for the validation set using the GridSearch optimized hyperparameters

val_grid_acc = accuracy_score(y_val, y_pred_val_grid) # Calculate the validation set accuracy score using the GridSearch optimized hyperparameters

print(f'Accuracy Score for GridSearch Validation Set is: {val_grid_acc}')

Accuracy Score for GridSearch Validation Set is: 0.8113636363636364


In [53]:
test_grid_pipeline = search_grid.best_estimator_ # Set up pipeline for test set using the GridSearch optimized hyperparameters

In [54]:
y_pred_test_grid = test_grid_pipeline.predict(X_test) # Predict target vector for the test set using the GridSearch optimized hyperparameters

submission_grid = pd.DataFrame(data = y_pred_test_grid, index = X_test.index, columns = ['status_group']) # Assign the predicted values to a dataframe
submission_grid.head()

,status_group
id,
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional


In [55]:
# Create Kaggle Submission file
submission_grid.to_csv('Ivan_Mihailov_Module3_Grid_Kaggle_Submission')